In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [3]:
dict_table = pickle.load(open('group_tables','rb'))
df_historical_data = pd.read_csv('football_all_matches_clean.csv')
df_fixture = pd.read_csv('WC2022_clean.csv')

<h1>TEAM STRENGTH</h1>

In [10]:
#splitting into df_home and df_away
df_home = df_historical_data[['Home','HomeGoals','AwayGoals']]
df_away = df_historical_data[['Away','HomeGoals','AwayGoals']]
df_away

,Away,HomeGoals,AwayGoals
0,Mexico,4,1
1,Argentina,4,2
2,Yugoslavia,6,1
3,United States,6,1
4,Belgium,1,0
...,...,...,...
1021,Costa Rica,2,0
1022,Switzerland,1,2
1023,Brazil,0,2
1024,Mexico,0,1


In [11]:
df_home.rename(columns={'Home':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'},inplace=True)
df_away.rename(columns={'Away':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'},inplace=True)

C:\Users\bibas\AppData\Local\Temp\ipykernel_7492\1604878116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_home.rename(columns={'Home':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'},inplace=True)
C:\Users\bibas\AppData\Local\Temp\ipykernel_7492\1604878116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_away.rename(columns={'Away':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'},inplace=True)


In [13]:
df_home

,Team,GoalsScored,GoalsConceded
0,France,4,1
1,Uruguay,4,2
2,Uruguay,6,1
3,Argentina,6,1
4,Paraguay,1,0
...,...,...,...
1021,Brazil,2,0
1022,Serbia,1,2
1023,Serbia,0,2
1024,Germany,0,1


In [20]:
#concat, groupby team, find mean
df_team_strength = pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,0.812500,1.312500
Angola,0.333333,0.666667
Argentina,1.711111,1.077778
Australia,0.954545,1.909091
Austria,1.482759,1.620690
...,...,...
Uruguay,1.532258,1.274194
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


<h1>PREDICTION POINT FUNCTION BY POISSON</h1>

In [31]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        lamb_home = df_team_strength.loc[home,'GoalsScored'] * df_team_strength.loc[away,'GoalsConceded']
        lamb_away = df_team_strength.loc[away,'GoalsScored'] * df_team_strength.loc[home,'GoalsConceded']

        prob_home, prob_draw, prob_away = 0,0,0
        for x in range(0,11):#Home Goals
            for y in range(0,11): #Away Goals
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y,lamb_away)

                if x==y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        home_points = 3 * prob_home + prob_draw
        away_points = 3 * prob_away + prob_draw
        return(home_points,away_points)
    else:
        return(0,0)


In [32]:
predict_points('Argentina','Saudi Arabia')


(2.854533717589766, 0.08601175965745106)

<h1>PREDICTION MATCHES</h1>

In [37]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [38]:
df_fixture_group_48

,Home,Score,Away,Year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [40]:
#group stage simul
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['Home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['Home'], row['Away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [42]:
dict_table['Group A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


In [43]:
df_fixture_knockout

,Home,Score,Away,Year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


<h1>KNOCKOUT</h1>

In [44]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,Home,Score,Away,Year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [47]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['Home'], row['Away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [48]:
get_winner(df_fixture_knockout)

,Home,Score,Away,Year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


<h1>Quarter Final</h1>

In [49]:
df_fixture_quarter

,Home,Score,Away,Year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [51]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'Score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [52]:
update_table(df_fixture_knockout, df_fixture_quarter)


,Home,Score,Away,Year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [53]:
get_winner(df_fixture_quarter)

,Home,Score,Away,Year,winner
56,Germany,Match 58,Brazil,2022,Germany
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


<h1>SEMI</h1>

In [54]:
update_table(df_fixture_quarter, df_fixture_semi)


,Home,Score,Away,Year,winner
60,Netherlands,Match 61,Germany,2022,?
61,France,Match 62,Portugal,2022,?


In [55]:
get_winner(df_fixture_semi)

,Home,Score,Away,Year,winner
60,Netherlands,Match 61,Germany,2022,Germany
61,France,Match 62,Portugal,2022,Portugal


<h1>FINAL</h1>

In [56]:
update_table(df_fixture_semi,df_fixture_final)

,Home,Score,Away,Year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Germany,Match 64,Portugal,2022,?


In [57]:
get_winner(df_fixture_final)

,Home,Score,Away,Year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Germany,Match 64,Portugal,2022,Germany


<h1>PREDICTED WINNERS: GERMANY</h1>